In [1]:
import os
import numpy as np
import cv2
import pickle
from tqdm import tqdm

# A function to extract one video's frames
def extract_one_video_frames(vid_path, det_dir, save_root):

    # Get the name of the video
    vname = os.path.basename(vid_path)
    country, city, vid = vname.split('+')

    # Create a directory to save the frames
    save_dir = os.path.join(save_root, country, city, vid[:-4])
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # Load the detection file
    det_file = os.path.join(det_dir, vname.replace('.mp4', '.pkl'))
    if not os.path.isfile(det_file):
        print(f'****** [Attention] detection file: {det_file} does not exist, skiped !!')
        return
    with open(det_file, 'rb') as f:
        dets = pickle.load(f)
    
    # Open the video using OpenCV
    cap = cv2.VideoCapture(vid_path)
    # Get the number of frames in the video
    nfs = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Sort the frame IDs
    frame_ids = sorted(dets.keys())

    # Loop through each frame
    for frame_id in frame_ids:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, image = cap.read()
        if not ret:
            print(f'****** [Attention] video: {vid_path} corrupt at frame: {frame_id} !!')
            continue
        # Loop through each detection in the frame
        for det in dets[frame_id]:
            obj_idx_at_ori_image = det[0]
            bbox = det[1]['bbox'].round().astype(np.int)
            pid = det[1]['pid']
            # Crop the image using the bounding box
            img = image[bbox[1]:bbox[3], bbox[0]:bbox[2], :]
            # Create a unique name for the image
            img_name = f'{pid:06d}_{frame_id:07d}_{obj_idx_at_ori_image:02d}.jpg'
            img_path = os.path.join(save_dir, img_name)
            # Save the image if it doesn't already exist
            if not os.path.exists(img_path):
                cv2.imwrite(img_path, img)


def get_all_videos(vid_dir):
    # Initialize an empty list to store the video file paths
    vids = []
    # Get a sorted list of directories in vid_dir that represent countries
    countries = sorted([x for x in os.listdir(vid_dir) if os.path.isdir(os.path.join(vid_dir, x))])
    # Iterate through the list of countries
    for country in countries:
        # Get the directory path for the current country
        country_dir = os.path.join(vid_dir, country)
        # Get a sorted list of directories in country_dir that represent cities
        cities = sorted([x for x in os.listdir(country_dir) if os.path.isdir(os.path.join(country_dir, x))])
        # Iterate through the list of cities
        for city in cities:
            # Get the directory path for the current city
            city_dir = os.path.join(country_dir, city)
            # Get a sorted list of video files with .mp4 extension in city_dir
            names = sorted([x for x in os.listdir(city_dir) if os.path.isfile(
                            os.path.join(city_dir, x)) and x.endswith('.mp4')])
            # Add the full path of the video files to the list of videos
            vids += [os.path.join(city_dir, x) for x in names]
    # Return the list of video file paths
    return vids

# Set the directory paths for the video files, detection files, and the save directory
vid_dir = "YOUR PATH/video"
det_dir = 'YOUR PATH/dets'
save_dir = 'YOUR PATH/people'
# Check if the directories exist and raise an error if not
if not os.path.exists(vid_dir):
    raise IOError(f'raw video directory: {vid_dir} does not exist')
if not os.path.exists(det_dir):
    raise IOError(f'detection file directory: {det_dir} does not exist')
if not os.path.exists(save_dir):
     # Create the save directory if it does not exist
    os.makedirs(save_dir)
# Get the list of video file paths
vids = get_all_videos(vid_dir)
# Iterate through the list of video file paths
for vid in tqdm(vids):
     # Call the extract_one_video_frames function for each video
    extract_one_video_frames(vid, det_dir, save_dir)

OSError: ignored